In [1]:
import os
import numpy as np
from scipy.ndimage import zoom

In [2]:
def nict_is(year, month, day):
    nlat, nlon, nmin = 30, 80, 1440
    maxlong_is, maxlat_is = 321, 221
    
    # Initialize arrays
    bispow = np.zeros((nlon, nlat, nmin))
    bisfac = np.zeros((nlon, nlat, nmin))
    bissxx = np.zeros((nlon, nlat, nmin))
    bissyy = np.zeros((nlon, nlat, nmin))
    bissxy = np.zeros((nlon, nlat, nmin))

    bigpow = np.zeros((nlon, nlat, nmin))
    bigfac = np.zeros((nlon, nlat, nmin))
    bigsxx = np.zeros((nlon, nlat, nmin))
    bigsyy = np.zeros((nlon, nlat, nmin))
    bigsxy = np.zeros((nlon, nlat, nmin))

    nnn = 0

    # String formatting
    mmdd = f"{month:02d}{day:02d}"
    yyyymmdd = f"{year}{month:02d}{day:02d}"

    # Directory setup
    dir_path = f"D:\\nict03\\{yyyymmdd}"
    os.chdir(dir_path)

    # Constants
    pi = np.pi
    fac_norm = 3.75e-6
    pow_norm = 1.017e7
    cond_norm = 1.50e01

    # Parameter initialization
    tew = np.zeros((maxlong_is, maxlat_is))
    fiw = np.zeros((maxlong_is, maxlat_is))
    vxw = np.zeros((maxlong_is, maxlat_is))
    vyw = np.zeros((maxlong_is, maxlat_is))
    vzw = np.zeros((maxlong_is, maxlat_is))
    pow = np.zeros((maxlong_is, maxlat_is))
    fac = np.zeros((maxlong_is, maxlat_is))
    sxx = np.zeros((maxlong_is, maxlat_is))
    syy = np.zeros((maxlong_is, maxlat_is))
    sxy = np.zeros((maxlong_is, maxlat_is))
    timea = np.zeros(1)
    nn = np.zeros(1, dtype=int)
    conbp = np.zeros(100)

    tew_fig = np.zeros(maxlat_is)
    fiw_fig = np.zeros(maxlong_is)

    x = np.zeros((maxlong_is, maxlat_is))
    y = np.zeros((maxlong_is, maxlat_is))
    z = np.zeros((maxlong_is, maxlat_is))

    for hour in range(24):
        for minute in range(60):
            hhmm = hour * 100 + minute
            if hour == 0 and minute == 0:
                yyyymmddhhmm = f"{yyyymmdd}0000"
            elif hour < 10:
                yyyymmddhhmm = f"{yyyymmdd}0{hhmm}"
            else:
                yyyymmddhhmm = f"{yyyymmdd}{hhmm}"

            print(yyyymmddhhmm, year, month, day, hour, minute)
            input_filename = f"idata{yyyymmddhhmm}"

            try:
                with open(input_filename, 'rb') as f:
                    nn = np.fromfile(f, dtype=np.int32, count=1)
                    timea = np.fromfile(f, dtype=np.float32, count=1)
                    vxw = np.fromfile(f, dtype=np.float32, count=maxlong_is * maxlat_is).reshape(maxlong_is, maxlat_is)
                    vyw = np.fromfile(f, dtype=np.float32, count=maxlong_is * maxlat_is).reshape(maxlong_is, maxlat_is)
                    vzw = np.fromfile(f, dtype=np.float32, count=maxlong_is * maxlat_is).reshape(maxlong_is, maxlat_is)
                    pow = np.fromfile(f, dtype=np.float32, count=maxlong_is * maxlat_is).reshape(maxlong_is, maxlat_is)
                    fac = np.fromfile(f, dtype=np.float32, count=maxlong_is * maxlat_is).reshape(maxlong_is, maxlat_is)
                    sxx = np.fromfile(f, dtype=np.float32, count=maxlong_is * maxlat_is).reshape(maxlong_is, maxlat_is)
                    sxy = np.fromfile(f, dtype=np.float32, count=maxlong_is * maxlat_is).reshape(maxlong_is, maxlat_is)
                    syy = np.fromfile(f, dtype=np.float32, count=maxlong_is * maxlat_is).reshape(maxlong_is, maxlat_is)

                    pow[0, 0] = pow[2, 0]
                    pow[1, 0] = pow[2, 0]
                    fac[0, 0] = fac[2, 0]
                    fac[1, 0] = fac[2, 0]

                pow2 = np.roll(pow[:320, :], -2, axis=0)

                bissxx[:, :, nnn] = zoom(sxx[:320, :60] * cond_norm, (nlon / 320, nlat / 60))
                bissxy[:, :, nnn] = zoom(sxy[:320, :60] * cond_norm, (nlon / 320, nlat / 60))
                bissyy[:, :, nnn] = zoom(syy[:320, :60] * cond_norm, (nlon / 320, nlat / 60))
                bispow[:, :, nnn] = zoom(pow2[:320, 1:61] * pow_norm, (nlon / 320, nlat / 60))
                bisfac[:, :, nnn] = zoom(fac[:320, :60] * fac_norm, (nlon / 320, nlat / 60))

                bigsxx[:, :, nnn] = zoom(sxx[:320, 161:221] * cond_norm, (nlon / 320, nlat / 60))
                bigsxy[:, :, nnn] = zoom(sxy[:320, 161:221] * cond_norm, (nlon / 320, nlat / 60))
                bigsyy[:, :, nnn] = zoom(syy[:320, 161:221] * cond_norm, (nlon / 320, nlat / 60))
                bigpow[:, :, nnn] = zoom(pow[:320, 161:221] * pow_norm, (nlon / 320, nlat / 60))
                bigfac[:, :, nnn] = zoom(fac[:320, 161:221] * fac_norm, (nlon / 320, nlat / 60))

                nnn += 1

            except FileNotFoundError:
                print(f"File {input_filename} not found, skipping...")

    # Save the final arrays (e.g., using numpy's save function or another method)
    np.save(f"{dir_path}/{yyyymmdd}_pss.npy", [bissxx, bissxy, bissyy, bispow, bisfac])
    np.save(f"{dir_path}/{yyyymmdd}_pnn.npy", [bigsxx, bigsxy, bigsyy, bigpow, bigfac])

